In [60]:
import requests

def get_nutrients(fdc_ids):
    # Initialize a dictionary to store the results
    results = {}

    # Iterate over the FoodData Central IDs
    for fdc_id in fdc_ids:
        # Define the API endpoint URL
        url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}"

        # Define the query parameters
        params = {"api_key": "B2Gehrdg2uN9J0Rk1o8PGrdRIL8ojBo3aaJOjxRb"}

        # Send the GET request
        response = requests.get(url, params=params)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Extract the JSON data from the response
            data = response.json()

            # Extract the food name
            food_name = data["description"]

            # Initialize the nutrients data dictionary with the food name
            nutrients_data = {"name": food_name}

            # Extract the nutrients data
            for nutrient in data["foodNutrients"]:
                nutrient_name = nutrient["nutrient"]["name"]
                nutrients_data[nutrient_name] = nutrient.get("amount", None)

            # Store the nutrients data in the results dictionary
            results[fdc_id] = nutrients_data
        else:
            print(f"Error: API request failed for FDC ID {fdc_id}")
            print("Status code:", response.status_code)
            print("Response text:", response.text)

    # Return the results
    return results

In [61]:
# Define the FoodData Central IDs
fdc_ids = ["748967", "170554", "2262072"]

# Call the function and store the results
results = get_nutrients(fdc_ids)

# Print the results in a readable format
pprint(results)

{'170554': {'Alanine': 1.044,
            'Amino acids': None,
            'Arginine': 2.143,
            'Ash': 4.8,
            'Aspartic acid': 1.689,
            'Calcium, Ca': 631.0,
            'Carbohydrate, by difference': 42.12,
            'Carbohydrates': None,
            'Cholesterol': 0.0,
            'Copper, Cu': 0.924,
            'Cystine': 0.407,
            'Energy': 2034.0,
            'Fatty acids, total monounsaturated': 2.309,
            'Fatty acids, total polyunsaturated': 23.665,
            'Fatty acids, total saturated': 3.33,
            'Fatty acids, total trans': 0.14,
            'Fiber, total dietary': 34.4,
            'Folate, food': 49.0,
            'Folate, total': 49.0,
            'Glutamic acid': 3.5,
            'Glycine': 0.943,
            'Histidine': 0.531,
            'Iron, Fe': 7.72,
            'Isoleucine': 0.801,
            'Leucine': 1.371,
            'Lipids': None,
            'Lysine': 0.97,
            'MUFA 14:1': 0.03,
    

In [77]:
import pandas as pd

# Convert the metadata dictionary to a DataFrame
df = pd.DataFrame.from_dict(results, orient="index")

# Add a duplicate index as a leading column called "index"
df["index"] = df.index

# Reset the index
df = df.reset_index(drop=True)

# Print the resulting DataFrame
display(df)

,name,Proximates,Water,Energy,Nitrogen,Protein,Total lipid (fat),Total fat (NLEA),Ash,Carbohydrates,...,SFA 24:0,MUFA 20:1 c,MUFA 22:1 n-9,Isoflavones,Daidzein,Genistein,Daidzin,Genistin,Glycitin,index
0,"Eggs, Grade A, Large, egg whole",None,75.800,617.0,1.990,12.40000,9.96,8.65,0.850,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,748967
1,"Seeds, chia seeds, dried",None,5.800,2034.0,NaN,16.54000,30.74,NaN,4.800,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,170554
2,"Peanut butter, creamy",None,1.104,NaN,4.394,23.99124,49.43,NaN,2.771,None,...,0.8243,0.8006,0.132,NaN,11.46,0.0,0.6493,1.932,0.4125,2262072


In [78]:
# Unpivot the DataFrame
df_unpivoted = pd.melt(df, id_vars=['index'], var_name='attribute', value_name='value')

# Drop rows with null values
df_unpivoted = df_unpivoted.dropna()

# Change the 'value' column to string type
df_unpivoted['value'] = df_unpivoted['value'].astype(str)

# Remove leading/trailing bracket and apostrophe
df_unpivoted['value'] = df_unpivoted['value'].str.strip("['']")

# Print the resulting unpivoted DataFrame
display(df_unpivoted)

,index,attribute,value
0,748967,name,"Eggs, Grade A, Large, egg whole"
1,170554,name,"Seeds, chia seeds, dried"
2,2262072,name,"Peanut butter, creamy"
6,748967,Water,75.8
7,170554,Water,5.8
...,...,...,...
383,2262072,Daidzein,11.46
386,2262072,Genistein,0.0
389,2262072,Daidzin,0.6493
392,2262072,Genistin,1.932


In [ ]:
df_unpivoted.to_csv('nutrition.csv', index=False)